In [42]:
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [44]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [45]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="web_collection",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
)

True

In [46]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="web_collection",
    embedding=embeddings,
)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
from langchain_community.document_loaders import PlaywrightURLLoader

urls = [
    "https://www.oraczen.ai"
]

loader = PlaywrightURLLoader(urls=urls, remove_selectors=["header", "footer"])
docs = await loader.aload()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [50]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [51]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [9]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    
    
    
    template = ("Use the following pieces of context to answer the question at the end."
"If you don't know the answer, just say that you don't know, don't try to make up an answer."
"Use three sentences maximum and keep the answer as concise as possible."
"Always say 'thanks for asking!' at the end of the answer."

f'Context: {docs_content}\n'

f'Question: {state["question"]}'
)
    
    system_msg = """ You are a customer service CHATBOT your name is 'ORACZEN AI'
    company details and service chatbot generate the relevent answer so clear in your responses. Be polite and helpful.
    If answer not found tell the user that the information is not available.
    
    
    """
    query_generate = ChatPromptTemplate.from_messages(
    [("system", system_msg), ("placeholder", "{template}")]
)
    output = query_generate | llm
    
    response = output.invoke({"template": [HumanMessage(content=template)]})
    return {"answer": response.content}

In [53]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [54]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile(checkpointer=memory)

In [57]:
result = graph.invoke({"question": "give me the contact details"}, config=config)
print(result)

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

{'question': 'give me the contact details', 'context': [Document(metadata={'source': 'https://shade6.com', '_id': 'a8c8efb3918444f085886a3e31288018', '_collection_name': 'web_collection'}, page_content='91501 34707\n\nEmail\n\nSuccess Stories\n\nFew of our works\n\nTestimonials\n\nWhat our clients talk about us!\n\nNeed more info?\n\nReach out to us so you can get the best!\n\nYour inquiry about * Web Application Development Mobile Application Development Website Creation BuildNow Platform Others\n\nReach out now!\n\n(+91) 91501 34707\n\nStart the collaboration with us while figuring out the best solution based on your needs.\n\nKonquest IT Park, Ambattur Industrial Estate\n\ninfo@shade6.com\n\n+91 91501 34707\n\nwww.shade6.com\n\nIT Services\n\nWeb Sites/Applications\n\nMobile Applications\n\nDesign\n\nE-Commerce\n\nQuick Links\n\nBuildNow\n\nAbout Us\n\nCareers\n\nSupport\n\nContact Us\n\nCase Studies\n\nBlogs\n\nLegal\n\nTerms of Use\n\nPrivacy Policy\n\n© 2023 Shade6. All Rights Re